In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
from collections import Counter
from sklearn.metrics import accuracy_score

In [ ]:
df = pd.read_csv(os.path.join("iris", "iris.data"), header=None)

classes = np.unique(df[4].values)
df[4] = df[4].map({'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2})

### Divide the dataset into training and validation

In [ ]:
def split_iris_data(df, training_split=0.5, random_state=None):
    setosa = df[df[4] == 0]
    versicolor = df[df[4] == 1] 
    virginica = df[df[4] == 2]

    # Randomly split each class into the training set
    train_set = pd.concat([setosa.sample(frac=training_split, random_state=random_state),
                            versicolor.sample(frac=training_split, random_state=random_state),
                            virginica.sample(frac=training_split, random_state=random_state)])

    # The remaining data composes the test set
    test_set = df.drop(train_set.index)

    # Shuffle both datasets
    train_set = train_set.sample(frac=1, random_state=random_state).reset_index(drop=True)
    test_set = test_set.sample(frac=1, random_state=random_state).reset_index(drop=True)
    
    # Make training targets and features
    train_features = train_set.drop(columns=[4]).values
    train_targets = train_set[4].values
    # Make test targets and features
    test_features = test_set.drop(columns=[4]).values
    test_targets = test_set[4].values

    return train_features, train_targets, test_features, test_targets

### Fisher's linear discriminator

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

class FisherLDA:
    """
    Fisher's LDA finds a projection direction w that maximizes the Fisher criterion:
    J(w) = (w^T * Sb * w) / (w^T * Sw * w)
    
    Where:
    - Sb is the between-class scatter matrix
    - Sw is the within-class scatter matrix
    - w is the projection vector we want to find
    
    The optimal w is found by solving the generalized eigenvalue problem:
    Sb * w = gamma * Sw * w
    Which is equivalent to: Sw^(-1) * Sb * w = gamma * w
    """
    
    def __init__(self, n_components=2):
        self.n_components = n_components
        self.W = None  
        self.class_means = None
        self.overall_mean = None
        self.Sw = None  
        self.Sb = None  
        
    def _calculate_class_means(self, X, y):
        classes = np.unique(y)
        class_means = {}
        for cls in classes:
            class_means[cls] = np.mean(X[y == cls], axis=0)
        return class_means
    
    def _calculate_within_class_scatter(self, X, y):
        classes = np.unique(y)
        n_features = X.shape[1]
        Sw = np.zeros((n_features, n_features))
        
        for cls in classes:
            class_data = X[y == cls]
            class_mean = self.class_means[cls]
            
            # Calculate scatter for this class: Σ(x - μ_i)(x - μ_i)^T
            class_scatter = np.zeros((n_features, n_features))
            for sample in class_data:
                diff = (sample - class_mean).reshape(-1, 1)
                class_scatter += diff @ diff.T
            
            Sw += class_scatter
            
        return Sw
    
    def _calculate_between_class_scatter(self, X, y):
        classes = np.unique(y)
        n_features = X.shape[1]
        Sb = np.zeros((n_features, n_features))
        
        for cls in classes:
            n_samples = np.sum(y == cls)
            class_mean = self.class_means[cls]
            
            # Calculate (μ_i - μ)(μ_i - μ)^T
            diff = (class_mean - self.overall_mean).reshape(-1, 1)
            Sb += n_samples * (diff @ diff.T)
            
        return Sb
    
    def fit(self, X, y):
        """ 
        Optimizes 
        max J(w) = (w^T * Sb * w) / (w^T * Sw * w) 
        by finding eigenvectors of Sw^(-1) * Sb
        """        
        self.overall_mean = np.mean(X, axis=0)
        self.class_means = self._calculate_class_means(X, y)
        
        # Calculate scatter matrices
        self.Sw = self._calculate_within_class_scatter(X, y)
        self.Sb = self._calculate_between_class_scatter(X, y)
        
        # Solve the generalized eigenvalue problem: Sw^(-1) * Sb * w = gamma * w
        Sw_inv = np.linalg.pinv(self.Sw)
        eigenvals, eigenvecs = np.linalg.eig(Sw_inv @ self.Sb)
        
        idx = np.argsort(eigenvals.real)[::-1]
        eigenvals = eigenvals[idx]
        eigenvecs = eigenvecs[:, idx]
        
        # Select the top n_components eigenvectors
        self.W = eigenvecs[:, :self.n_components].real
        
        return self
    
    def transform(self, X):
        return X @ self.W
    
    def fit_transform(self, X, y):
        return self.fit(X, y).transform(X)

def one_vs_rest_lda(X_train, y_train, X_test, y_test, plot=True):
    """
    Perform one-vs-rest Fisher's LDA for each class
    
    For each class i:
    1. Create binary problem: class i vs all other classes
    2. Apply Fisher's LDA to find optimal projection using training data
    3. Evaluate classification performance on test data
    """
    classes = np.unique(y_train)
    results = {}
    
    for target_class in classes:        
        # Create binary labels for train and test: 1 for target class, 0 for others
        y_train_binary = (y_train == target_class).astype(int)
        y_test_binary = (y_test == target_class).astype(int)
        
        # Apply Fisher's LDA with n_components=1 on training data
        lda = FisherLDA(n_components=1)
        X_train_proj = lda.fit_transform(X_train, y_train_binary)
        
        # Transform test data using fitted LDA
        X_test_proj = lda.transform(X_test)
        
        # Calculate Fisher's criterion value
        fisher_criterion = (lda.W.T @ lda.Sb @ lda.W) / (lda.W.T @ lda.Sw @ lda.W)
        
        # Calculate threshold using midpoint of projected class means
        mean_target_proj = lda.class_means[1] @ lda.W.flatten()  
        mean_rest_proj = lda.class_means[0] @ lda.W.flatten()    
        threshold = (mean_target_proj + mean_rest_proj) / 2
        
        target_above_threshold = mean_target_proj > threshold        
        if target_above_threshold:
            # Standard case: target class projects to values > threshold
            predictions_one = X_test_proj[y_test_binary == 1].flatten() > threshold
            predictions_rest = X_test_proj[y_test_binary == 0].flatten() <= threshold
        else:
            # Flipped case: target class projects to values < threshold
            predictions_one = X_test_proj[y_test_binary == 1].flatten() < threshold
            predictions_rest = X_test_proj[y_test_binary == 0].flatten() >= threshold

        # Calculate accuracy
        correct_predictions = np.sum(predictions_one) + np.sum(predictions_rest)
        total_samples = len(predictions_one) + len(predictions_rest)
        accuracy = correct_predictions / total_samples

        results[target_class] = {
            'accuracy': accuracy,
        }
        
        # Plot results using test data
        if plot:
            plt.figure(figsize=(12, 5))
            
            # Plot 1: Original test data
            plt.subplot(1, 2, 1)
            plot_df_orig = pd.DataFrame({
                'Feature 1': X_test[:, 0],
                'Feature 2': X_test[:, 1], 
                'Class': ['Target Class' if label == 1 else 'Other Classes' for label in y_test_binary]
            })
            sns.scatterplot(data=plot_df_orig, x='Feature 1', y='Feature 2', hue='Class', 
                            palette={'Other Classes': 'red', 'Target Class': 'blue'}, alpha=0.7)
            plt.title(f'Test Data - Class {target_class} vs Rest')
            # Plot 2: LDA projections of test data
            plt.subplot(1, 2, 2)
            plot_df_lda = pd.DataFrame({
                'LDA Projection': X_test_proj.flatten(),
                'Class': ['Target Class' if label == 1 else 'Other Classes' for label in y_test_binary],
                'Index': range(len(X_test_proj.flatten()))
            })
            
            sns.scatterplot(data=plot_df_lda, x='Index', y='LDA Projection', hue='Class', 
                            palette={'Other Classes': 'red', 'Target Class': 'blue'}, 
                            alpha=0.7, s=60)
            
            # Add decision boundary
            plt.axhline(threshold, color='green', linestyle='--', linewidth=2, label='Decision boundary')
            plt.title(f'LDA Projection - Class {target_class} vs Rest\nTest Accuracy: {accuracy:.2%}')
            plt.xlabel('Sample Index')
            plt.ylabel('LDA Projection')
            plt.legend(loc='upper right')
            plt.grid(True, alpha=0.3)
            plt.show()

    return results

### One vs. Rest Analysis

In [ ]:
# Load and prepare the data
X_train, y_train, X_test, y_test = split_iris_data(df, training_split=0.5)

# Perform one-vs-rest LDA
results = one_vs_rest_lda(X_train, y_train, X_test, y_test)

# Summary
for cls, result in results.items():
    print(f"Class {cls}: Accuracy = {result['accuracy']:.4f}")

In [ ]:
# -- Fisher's LDA One-vs-Rest Implementation (20 runs) --
all_runs_results = []

for run in range(20):  
    # Split the data for this run (no random_state for variability)
    X_train, y_train, X_test, y_test = split_iris_data(df, training_split=0.5)
    
    run_results = {}
    
    # Loop through each class to perform one-vs-rest classification
    for i, class_name in enumerate(classes):
        
        # Prepare Data for One-vs-Rest 
        y_train_ovr = np.where(y_train == i, 1, 0)
        y_test_ovr = np.where(y_test == i, 1, 0)

        X_train_one = X_train[y_train_ovr == 1]
        X_train_rest = X_train[y_train_ovr == 0]
        
        # Calculate Mean Vectors
        mean_one = np.mean(X_train_one, axis=0)
        mean_rest = np.mean(X_train_rest, axis=0)

        # Calculate Scatter matrices and the projection vector w
        s_one = (X_train_one - mean_one).T @ (X_train_one - mean_one)
        s_rest = (X_train_rest - mean_rest).T @ (X_train_rest - mean_rest)
        Sw = s_one + s_rest
        Sw_inv = np.linalg.inv(Sw)
        w = Sw_inv @ (mean_one - mean_rest)

        # Project Data and Find Threshold 
        X_test_one = X_test[y_test_ovr == 1]
        X_test_rest = X_test[y_test_ovr == 0]

        projected_one = X_test_one @ w
        projected_rest = X_test_rest @ w

        # A simple threshold is the midpoint of the projected means
        threshold = (mean_one @ w + mean_rest @ w) / 2
        predictions_one = projected_one > threshold
        predictions_rest = projected_rest > threshold
        
        # Calculate accuracy
        correct_predictions = np.sum(predictions_one == 1) + np.sum(predictions_rest == 0)
        total_samples = len(projected_one) + len(projected_rest)
        accuracy = correct_predictions / total_samples
        
        run_results[class_name] = accuracy
    
        all_runs_results.append(run_results)

for i, class_name in enumerate(classes):
    accuracies = [run[class_name] for run in all_runs_results]
    mean_acc = np.mean(accuracies)
    var_acc = np.var(accuracies)
    min_acc = np.min(accuracies)
    max_acc = np.max(accuracies)
    
    print(f"\n{class_name} vs. Rest:")
    print(f"  Mean Accuracy: {mean_acc:.6f}")
    print(f"  Variance: {var_acc:.6f}")